# Teil d - Merkmalstandardisierung

# Verbesserung durch Merkmalstandardisierung

## Methode der Standardisierung und Auswirkung auf die Algorithmen

Viele Lernalgorithmen machen es erforderlich die Merkmale zu standardisieren um eine optimale Leistung zu erzielen. Die Algorithmen Perzeptron und Adaline gehören zu den vielen Algorithmen, die von einer Standardisierung profitieren.

Im folgenden wird die Methode <i>Standardisierung</i> erläutert. Diese verleiht den Daten die Eigenschaften einer Standardnormalverteilung. Der Mittelwert jedes Merkmals beträgt 0, die Standardabweichung jeder Spalte beträgt 1. Um zum Beispiel das Merkmal j zu standardiesieren, wird der Mittelwert $\mu$ der jeweiligen Stichprobe abgezogen und das Ergebnis durch die Standardabweichung $\sigma$ dividiert. Das Standardisierungsverfahren wird auf alle Merkmale der Datenmenge angewendet.

$x_j^{\prime(i)} = \frac{x_j^{(i)} - \mu_j}{\sigma_j}$. 

Die Standardisierung verbessert die Algorithmen, weil zum Auffinden einer guten/ optimalen Lösung (das globale Minimum der Straffunktion) weniger Schritte erforderlich sind. Folgende Abbildung stellen die Strafflächen einer zweidimensionalen Klassifizierungsaufgabe als Funktion der Gewichtungen dar.

<img src="./Figures/Merkmalstandardisierung.png" alt="drawing" style="width:500px;"/>


## Implementierung

Selektieren Sie exakt dieselben Daten des Iris-Datensatzes aus Teil a und nehmen Sie die Standardisierung vor. <br>

Trainineren Sie den entweder den Perzeptron-Algorithmus oder den Adaline-Algorithmus aus Teil a mit den standardisierten Daten mit verschiedenen Parametern. <br>

Vergleichen Sie die Ergebnisse zwischen den standardisierten Daten und den nicht-standardisierten Daten. Stellen Sie die verschiedenen Resultate dar.<br>

## Auswahl der Daten

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

In [5]:
# TODO Implement
df = pd.read_csv("./Data/iris.data", header=None, sep=",")

# Auswahl von setosa und versicolor
y = df.loc[((df[4].str.contains('setosa')) | (df[4].str.contains('versicolor')) ), 4].map({'Iris-setosa': 0, 'Iris-versicolor': 1})

# Auswahl von Kelch- und Bluetenblattlaenge
X = df.loc[y.index, [0, 2]]
y = np.array(y)

## Implementierung der Standardisierung

In [8]:
def normalize(df):
    df_normalized = df.copy()
    
    for col_name in df.columns:

        u = np.mean(df_normalized.loc[:,col_name])
        var = np.var(df_normalized.loc[:, col_name])
        o = np.sqrt(var)
        print("{}:\n---\nMittelwert: {}\nVarianz: {}\nStandartabweichung: {}".format(col_name, u, var, o))

        for idx, val in enumerate(df_normalized.loc[:,col_name]):           
            df_normalized.loc[idx, col_name] =  (val - u )/o

        u_strich = 1/o*(u-u) # siehe wikipedia
        var_strich = np.var(df_normalized[col_name])
        o_strich = np.sqrt(var_strich)
        print("Mittelwert' : {}\nVarianz' : {}\nStandartabweichung' : {}\n".format(u_strich, var_strich, o_strich))
    return df_normalized

In [11]:
X_std = normalize(X)

0:
---
Mittelwert: 5.471000000000001
Varianz: 0.40765900000000016
Standartabweichung: 0.6384817930058775
Mittelwert' : 0.0
Varianz' : 0.9999999999999994
Standartabweichung' : 0.9999999999999997

2:
---
Mittelwert: 2.8620000000000005
Varianz: 2.0773560000000004
Standartabweichung: 1.4413035766277695
Mittelwert' : 0.0
Varianz' : 0.9999999999999997
Standartabweichung' : 0.9999999999999998



## Training und Visualisierung des Errors

In [12]:
class Perceptron(object):
    
    def __init__(self, eta=0.01, epochs=10):
        self.eta = eta 
        self.epochs = epochs
        
    def gewichtete_summe(self, x):
        print(x, self.w[1:])
        weightedSum = self.w[1:].T.dot(x) + self.w[0]
        print(f'Weighted Sum: {weightedSum}')
        return weightedSum
        
    def heaviside(self, summe):
        if summe >= 0:
            return 1
        else:
            return 0
    
    def fit(self, X, y):
        errors_per_Epoch = []
        total_Errors = []
        numberOfFeatures = X.shape[1]
        weights = self.initWeights(numberOfFeatures)
        self.w = np.insert(weights,0,[1],axis = 0)
        print(f'Initial Weights: {self.w}')
        for epoch in range(self.epochs):
            print(f"Epoch {epoch}")
            for idx, x in enumerate(X.values): 
                y_pred = (self.heaviside(self.gewichtete_summe(x)))
                errors_per_Epoch.append(self.calculateError(y[idx], y_pred))
                for feature in range(numberOfFeatures):
                    self.updateWeights(y[idx], y_pred, feature, x[feature])
                    
            total_Errors.append(np.array(errors_per_Epoch).sum())       
            errors_per_Epoch = []         

        return total_Errors

    def initWeights(self, size):
        mu, sigma = 0, 0.01
        return np.random.normal(mu, sigma, size)
    
    def updateWeights(self, y_true, y_pred, weightIndex, xj):
        print(f'Old Weight: {self.w[weightIndex+1]}')
       
        weightfactor = xj * (self.eta*(y_true - y_pred)) 
        self.w[weightIndex+1] += weightfactor
        print(f'New Weight: {self.w[weightIndex+1]}')
        
    def calculateError(self, y_true, y_pred):
        return abs(y_true - y_pred)

In [ ]:
# TODO: Implement
list_of_epochs = [10,20,50,100]
list_of_etas = [1,0.1, 0.01, 0.00001, 0.0000001]

for epochs in list_of_epochs:
    for eta in list_of_etas: 
    
        perceptron = Perceptron(eta=eta, epochs=epochs)
        error = perceptron.fit(X_std,y)
        
        plt.plot(range(epochs), error)
        plt.ylabel("Sum of false classfication per epoch")
        plt.xlabel("Epochs")
        plt.title("Epochs " + str(epochs) + " Learning Rate" + str(eta))
        plt.savefig("./Figures/TeilD/" + str(epochs) + "_" +  str(eta).replace(".","") + ".png")
        plt.show()